## Quality control/Exploratory data analysis Notebook

By: Megan Grout (groutm@ohsu.edu)

Adapted from code written by Dr. Marilyne Labrie and Nick Kendsersky


Last updated: 20191219

Import external libraries.

In [ ]:
import os
import random
import re
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import subprocess


from scipy import signal

import plotly.figure_factory as ff
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
import plotly.express as px
init_notebook_mode(connected = True)

Import function written for this project.

In [ ]:
from cycif_modules import *

Define function to change header names. Not encapsutated in `cycif_modules`, so that user can change on the fly as necessary.

In [ ]:
# This may change for each experiment, so I have not sequestered
# this code in the cycif_modules.py file

# This function takes in a dataframe, changes the names
# of the column in various ways, and returns the dataframe.
# For best accuracy and generalizability, the code uses
# regular expressions (regex) to find strings for replacement.
def apply_header_changes(df):
    # remove lowercase x at beginning of name
    df.columns = df.columns.str.replace("^x","")
    # remove space at beginning of name
    df.columns = df.columns.str.replace("^ ","")
    # replace space with underscore
    df.columns = df.columns.str.replace(" ","_")
    # fix typos
    df.columns = df.columns.str.replace("CKD1","CDK1")
    df.columns = df.columns.str.replace("GAG3","GATA3")
    return df

## Begin Workflow

### Get directories

In [ ]:
# Base directory for project
base_dir = '/Users/groutm/Desktop/weewin'
base_dir = '/Users/groutm/Desktop/reproducibility'
base_dir = 'Z:\Marilyne\Axioscan\Gao_Zhang\Segmentation'
base_dir = '/Users/groutm/Desktop/gz_new'

# Set name for of project
# for use in directory creation
project_name = 'ww'
project_name = 'repro'
project_name = 'gz_new'

# Set string for current step, and for previous step
# for use in file and direcotry naming
step_suffix = 'bs'
previous_step_suffix_long = "_qc_eda"

# Initial input data directory
#input_data_dir = r'/Users/groutm/Desktop/TMAdata'
#input_data_dir = r'/Users/groutm/Desktop/ww_data'
input_data_dir = os.path.join(base_dir, project_name + previous_step_suffix_long)


# BS directory
#output_data_dir = r'/Users/groutm/Desktop/TMAoutputdata'
#output_data_dir = r'/Users/groutm/Desktop/ww_outputdata'
output_data_dir = os.path.join(base_dir, project_name + "_" + step_suffix)

# BS images subdirectory
#output_images_dir = r'/Users/groutm/Desktop/TMAimages'
#output_images_dir = r'/Users/groutm/Desktop/wwimages'
output_images_dir = os.path.join(output_data_dir,"images")

# Metadata directories
metadata_dir = os.path.join(base_dir, project_name + "_metadata")
metadata_images_dir = os.path.join(metadata_dir,"images")

# Create necessary directories for this step, if they don't already exist
for d in [base_dir, input_data_dir, output_data_dir, output_images_dir, 
          metadata_dir, metadata_images_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

# Change directory to location of input files
os.chdir(input_data_dir)



Create list of samples for use in this step of workflow. Do not include file extensions or steps labels.

In [ ]:
# Provide list of samples whose files we want to read int
# Needs to be a list of strings, which serve as bases for 
# input file names. Input files will be derived from base
# sample names, previous step substring, and filetype 
# extension

ls_samples = ['TMA','ww1', 'ww10', 'ww11', 'ww12', 'ww13', 'ww15', 
              'ww16', 'ww17', 'ww19', 'ww2', 'ww20', 'ww21', 
              'ww22', 'ww23', 'ww3', 'ww4', 'ww5', 'ww6', 'ww7', 
              'ww8', 'ww9']

ls_samples = ['TMA1.1', 'TMA1.2', 'TMA1.3', 'TMA2.1', 'TMA2.2', 'TMA2.3']

ls_samples = ['GZ10.1', 'GZ10.2', 'GZ10.3', 'TMA',
 'GZ7.1', 'GZ6', 'GZ7.2']

ls_samples = ['A_GZ2', 'B_GZ1', 'C_GZ5', 'D_GZ4', 'E_GZ3','F_GZ6','G_GZ7', 'H_GZ9','I_GZ10','TMA']

List of columns that are not marker intensities. It is okay if any of these are not actually present in a given dataframe. 

In [ ]:
not_intensities = ['replicate_ID', 'cell_type', 'Nucleus_Roundness', 'Nucleus_Size', 'Cell_Size',
                   'Nuc_X', 'not','Nuc_X_Inv','Cell_ID','Nuc_Y_Inv','ROI_slide','ROI_index','Nuc_Y',
                   'cluster']

## Import segmentation files

First, ascertain header of first sample's input file. This information will be used as a template against which all other input data files' headers will be tested.

In [ ]:
# Read in the first row of the file correpsonding to the first sample (index = 0)
# in ls_samples

# We do not need to specify a directory, since we earlier changed
# the current working directory to be that containing these files
filename = ls_samples[0] + previous_step_suffix_long + ".csv"

# Read in only the first line
df = pd.read_csv(filename, index_col = 0, nrows = 1)

# Verify that the ID column in input file became the index
# For segmentation files, we need the first column to be the 
# cell index. In later steps, the cell index will actually not
# be a proper dataframe data column, but the index of the saved
# dataframe from the previous step.
if df.index.name != "ID":
    print("Expected the first column in input file (index_col = 0) "
         "to be 'ID'. This column will be used to set the index names"
         "(cell number for each sample). It appears that the column '"
         + df.index.name + "' was actually the imported as the index "
         "column.")

# Apply the changes to the headers as specified in above funciton
df = apply_header_changes(df)

# Set variable to hold default header values
expected_headers = df.columns.values

For this entry point into the workflow, we expect the first column to be the ID index.

In [ ]:
df.index.name

FYI - What are the headers in our dataframe?

In [ ]:
print("Used " + ls_samples[0] + ".csv to determine the expected, corrected headers for all files.")
print("There headers are: \n" + ", ".join([h for h in expected_headers]) + ".")

#### Import segmentation files for analysis

In [ ]:
# Set dictionary to hold all individual sample data
dfs = {}

# iterate through each sample in our list of samples
for sample in ls_samples:
    # open the file
    # set the index to be the first (0-based indexing, so 0th)
    # column in input file.
    df = pd.read_csv('{}.csv'.format(sample), index_col = 0)#,
                    #nrows = 500) 
    # use nrows = # to specify number of input rows if you want
    
    # Check for empty df
    # if so, don't continue trying to process df
    if df.shape[0] == 0:
        print('Zero content lines detected in ' + sample + ' file.'
              'Removing from analysis...')
        # Remove from list, so further steps won't be looking
        # for data on this sample.
        # Note that for lists, we do not need to re-assign
        # the list when removing an item, i.e., we do not say
        # 'ls_samples = ls_samples.remove(sample)', since this
        # operation does not return anything.
        ls_samples.remove(sample)
        continue
    
    
    # Verify that the loaded df are the right length
    # commenting out because this code did not work on all
    # machines during testing (failed one PC, succeeded with
    # one PC and one MacBook)
    try:
        verify_line_no(sample + ".csv", df.shape[0] + 1) 
    except:
        pass
    # adding 1 because we expect the header was detected 
    # during file import and not counted towards length of df
    
     # Manipulations necessary for concatenation
    df = apply_header_changes(df)
    # sort them alphanetically
    df = df[[x for x in sorted(df.columns.values)]]
    
    
    # Compare headers of new df against what is expected
    compare_headers(expected_headers, df.columns.values, sample)
    
    # Add Sample_ID column and set it equal to sample name for sample
    df['Sample_ID'] = sample
    
    
    
    # For cases where we have samples called TMA1.1, TMA1.2, TMA1.3, etc.
    # Using regular expressions (regex) to extract the characters in the
    # sample name from TMA to the following digits, stopping at the period
    #if 'ROI_index' in df.columns.values:
     #   df['ROI_slide'] = re.findall(r'(TMA\d+)',sample)[0]
        
    # Add to dictionary of dfs 
    dfs[sample] = df
    



#Merge dfs into one big df
df = pd.concat(dfs.values(), ignore_index=False , sort = False)
# remove dfs from memory, since its big (relatively) and we
# don't need a data struture of all samples' data separated
# individually when we can extract information from the big
# df using the Sample_ID column
del dfs

# set index to Sample_ID + cell number
df = df.copy().reset_index(drop=True)
index = []
# Iterate through each sample, and extract from the big
# df just the rows corresponding to that sample. Then, 
# reassign the cell index based off of the Sample_ID value
# and the row number within that chunk. Save that information
# in a list of indices
for sample in ls_samples:
    df_chunk = df.loc[df['Sample_ID'] == sample,:].copy()
    old_index = df_chunk.index
    df_chunk = df_chunk.reset_index(drop=True)
    df_chunk = df_chunk.set_index(f'{sample}_Cell_' + df_chunk.index.astype(str))
    index = index + df_chunk.index.values.tolist()

# Use our list of indices to reassign the big df index
df.index =  index
# Remove the 'level_0' and 'index' columns that resulted
# from the above steps. This is not removing the actual index
# of the df, just a data column CALLED index.
df = df.loc[:,~df.columns.isin(['level_0','index'])]

Let's take a look at a few features to make sure our dataframe is as expected. We want to make sure the data import and aggregation steps worked well.

In [ ]:
df.index

In [ ]:
df.shape

Check for NaN entries (should not be any unless columns do not align), which can result from stitching together dfs with different values in their headers.

In [ ]:
# if there are any null values, then print names of columns containing
# null values
if df.isnull().any().any():
    print(df.columns[df.isnull().any()])

#in 'if' statement, false means no NaN entries True means NaN entries 

Check that all expected files were imported into final dataframe by comparing our sample names to the unique values in the Sample_ID column.

In [ ]:
if sorted(df.Sample_ID.unique()) == sorted(ls_samples):
    print("All expected filenames present in big df Sample_ID column.")
else:
    compare_headers(['no samples'], df.Sample_ID.unique(), "big df Sample_ID column")

List of header values that are not intensities. Can include items that aren't in a given header.

Need to save `not_intensities` list for future reference.

In [ ]:
fn = os.path.join(metadata_dir,"not_intensities.csv")

# If this file already exists, add only not_intensities items not already present in file
if os.path.exists(fn):
    print("'not_intensities.csv' already exists.")
    print("Reconciling file and Jupyter notebook lists.")
    # Open file as read-only, extract data
    fh = open(fn, "r")
    file_ni = fh.read().splitlines()
    # Set difference to identify items not already in file
    to_add = set(not_intensities) - set(file_ni)
    # We want not_intensities to the a complete list
    not_intensities = list(set(file_ni) | set(not_intensities))
    fh.close()
    # Open file for appending, writing new items
    fh = open(fn, "a")
    for item in to_add:
        fh.write(item +"\n")
    fh.close()
    
# The file does not yet exist
else:
    print("Could not find " + fn + ". Creating now.")
    # Open file for writing (will over-write exisiting file),
    # write all items
    fh = open(fn, "w")
    for item in not_intensities:
        fh.write(item + "\n")
    fh.close()

### Drop unwanted columns

Here, we are dropping a number of columns that we are totally uninterested in. For example, in the current workflow of QI Tissue, we can either export all columns (all markers in all cell components--cell, nucleus, cytoplasm) or individually check each and every one we want. It is faster and easier for the user, and maybe less error-prone, to export all columns and then drop those we are unintersted in here. Not every marker is expected to express in every location; this is why we might drop certain columns. Likewise, we may only be intersted in Average intensity in some features and Maximum intensity in others.

In [ ]:
# For development purposes, we kept all marker columns in the Cell and that were Intensity Averages.
# So the columns we want to keep:
# not_intensities, and any intensity column that contains 'Intensity_Average'
# We will be listing those columns we want to keep. Alternatively, you could name the columns you want to drop,
# or a mixture of both tactics.

# To get the 'Intensity_Average' columns, we use list comprehension:
# first get a list of all df columns not in 'not_intensities', aka, 
# those that ARE intensities, 'x for x in df....'
# Then, we only include them if they contain 'Intensity_Average',
# "...if 'Intensity_Average' in x"

## Explain how to add more, beyond Cell_Intensity_Average, etc.

to_keep = not_intensities \
    + [x for x in df.columns.values[~df.columns.isin(not_intensities)] if 'Cell_Intensity_Average' in x]

# If there are more columns we want to keep, we could include them by
# adding them to our 'to_keep' list
# to_keep.append(another_column)
# NOTE - do NOT reassign this to to_keep (to_keep = to_keep.append(item)),
# since the return value is None, for some reason. So you would be saying:
# to_keep = to_keep.append(item)
# to_keep = None
# to_keep --> would display 'None'
# to _keep = to_keep + [list, of, columns]
# here, you DO ressign (list = list + other_list)

# In order to extract only the columns we want from our big df, 
# we need to only ask for those that are IN the df.
# Our to_keep list contains items that might not be in our df headers!
# These items are from our not_intensities list. So let's ask for only those items
# from to_keep that are actually found in our df
df = df[[x for x in to_keep if x in df.columns.values]]


# What if we want to drop certain markers by name?
# Drop specific markers
#df = df.drop(columns = [])

Let's take a look at column names to make sure they are as expected.

In [ ]:
df.columns.values

### Nucleus size analysis

#### Distribution plots

In [ ]:
# Plot only cells where nucleus_size is [0, 500]
make_distr_plot_per_sample(
    title = "Initial dataframe nucleus sizes - 500 cutoff",
    location = output_images_dir, dfs = [df], 
    df_names = ["Initial dataframe"], colors = ["blue"], 
    x_label = "Nucleus Size", 
    legend = False, xlims = [0,500], markers = ['Nucleus_Size'])


In [ ]:
# Plot only cells where nucleus_size is [0, 100]
make_distr_plot_per_sample(title = "Initial dataframe nucleus sizes to 100",
                           location = output_images_dir, dfs = [df], 
                           df_names = ["Initial dataframe"], colors = ["blue"], 
                           x_label = "Nucleus Size", 
                           legend = False, xlims = [0,100], markers = ['Nucleus_Size'])


#### Peak analysis

Find valleys between peaks in nucleus size data - unfinished, but left here in case it aids future development.

In [ ]:
# Unfinished, but could consider using the following function
m = signal.find_peaks(df["Nucleus_Size"], prominence = 10, threshold = 20)
m[0].shape

#### Quantiles

Get quantiles (5th, 50th, 95th)

In [ ]:
qs = [0.05,0.50,0.95] # list of nucleus size percentiles to extract 
# Extract quantiles
nuc_sizes = pd.DataFrame(df["Nucleus_Size"].quantile(q=qs))
nuc_sizes['quantiles'] = nuc_sizes.index
nuc_sizes = nuc_sizes.reset_index().drop(columns = ['index'])

# Display df
nuc_sizes
## Save these data to file
filename = "nuc_quantile_sizes.csv"
filename = os.path.join(output_data_dir,filename)
nuc_sizes.to_csv(filename, index = False)

#### Nucleus size and other feature scatter plot

Scatter plot – to be most informative, ideally this would be cell size vs nucleus size, where color = nucleus roundness. Not all data used to develop workflow had all necessary features, so the actual data plotted below may not be terribly useful.


In [ ]:
# Set string variables
title = "Nucleus size by cell size for initial dataframe"
x_label = "Cell Size"
y_label = "Nucleus Size" # cell size - weewin data only has Nuc size!

# Create figure
fig = px.scatter(df, x="Cell_Size", y="Nucleus_Size",
                 color='Nucleus_Roundness')

#  Update layout for the aesthetic parameters we want
fig.update_layout(title_text=title, font=dict(size=18), 
        plot_bgcolor = 'white', showlegend = True )
# Adjust opacity
fig.update_traces(opacity=0.6)
# Adjust x-axis parameters
fig.update_xaxes(title_text = x_label, showline=True, linewidth=2, linecolor='black', 
        tickfont=dict(size=18))
    # Adjust y-axis parameters
fig.update_yaxes(title_text = y_label, showline=True, linewidth=2, linecolor='black',
        tickfont=dict(size=18))

# Display plot
#plot(fig)
filename = os.path.join(output_images_dir, title.replace(" ","_") + ".png")
fig.write_image(filename)

### Delete columns as necessary

Move forward with only the columns of interest

In [ ]:
# Remove columns containing "DAPI"
# use list comprehension to extract only column headers
# that do not contain the string "DAPI"
df = df[[x for x in df.columns.values if 'DAPI' not in x]]

print("Columns are now...")
print([c for c in df.columns.values])

### Create lists of full names and shortened names to use in plotting

We want a list of shortened marker intensity column header values for use in plotting. For example 'pATR_Cell_Intensity_Average' would display as 'pATR' for readability. In the case of more than one column present for a given marker, e.g., the inclusion of 'pATR_Nucleus_Cell_Intensity_Average', the pltoted labels would be 'pATR_Cell' and 'pATR_Cell'. We want to create dictionaries of both full to short names and short to full names.

In [ ]:
full_to_short_names, short_to_full_names =  \
    shorten_feature_names(df.columns.values[~df.columns.isin(not_intensities)])

Save this data to a metadata file. These devices will be used throughout the workflow.

In [ ]:
filename = os.path.join(metadata_dir, "full_to_short_column_names.csv")
fh = open(filename, "w")
fh.write("full_name,short_name\n")
for k,v in full_to_short_names.items():
    fh.write(k + "," + v + "\n")
    
fh.close()

In [ ]:
filename = os.path.join(metadata_dir, "short_to_full_column_names.csv")
fh = open(filename, "w")
fh.write("short_name,full_name\n")
for k,v in short_to_full_names.items():
    fh.write(k + "," + v + "\n")
    
fh.close()

In [ ]:
## Print contents to screen if the user wants

#for key, value in full_to_short_names.items():
#    print(key + ": " + value)

### Import exposure time metadata

Here, we want to end up with a data structure that incorporates metadata on each intensity marker column used in our big dataframe in an easy-to-use format. This is going to include the full name of the intensity marker columns in the big data frame, the corresponding round and channel, the target protein (e.g., CD45), and the segmentation localization information (cell, cytoplasm, nucleus)... We can use this data structure to assign unique colors to all channels and rounds, for example, for use in later visualizations.

Here, we expect this exposure time metadata file to have four columns (more are accepted). These are as follows:

- Round: The round in which the marker was assess. Should be in form 'r#'
- Target: The target/marker used. This should be a string whose contents match in the imported segmentation data files. The capitalization does not need to be consistent. These values should be unique in this file, without duplicates.
- Exp: The exposre time for this marker for this channel, in milliseconds. Not currently used in workflow.
- Channel: THe channel in which the marker was assessed. Should be in form 'c#'.


In [ ]:
filename = "Exposure_Time.csv"
#filename = "Exposure_Time_full.csv"
filename = os.path.join(metadata_dir, filename)


exp_df = pd.read_csv(filename)

In [ ]:
# Verify file imported correctly

# This part is wrapped in a try/except block because 
# it wasn't working on the PC workstation, but worked
# on MG's personal PC laptop and department loaner MacBook
try:
    verify_line_no(filename, exp_df.shape[0] + 1)
    print("Ran file size verification.")
except:
    pass

# Headers
print("Assessing whether column headers are as expected.")
expected_headers =['Round','Target','Exp','Channel']
compare_headers(expected_headers, exp_df.columns.values, "Imported metadata file")

# Missingness
if exp_df.isnull().any().any():
    print("\nexp_df has null value(s) in row(s):")
    print(exp_df[exp_df.isna().any(axis=1)])
else:
    print("No null values detected.")

Check to make sure that there are not duplicate values in the Target column.

In [ ]:
if len(exp_df['Target']) > len(exp_df['Target'].unique()):
    print("One or more non-unique Target values in exp_df. Currently not supported.")

In [ ]:
exp_df.sort_values(by = ['Target']).head()

In [ ]:
# Create lowercase version of target
exp_df['target_lower'] = exp_df['Target'].str.lower()
exp_df.head()

Create dataframe that contains marker intensity columns in our df that aren't in `not_intensities`

In [ ]:
intensities = pd.DataFrame({'full_column':df.columns.values[~df.columns.isin(not_intensities)]})

In [ ]:
intensities.head()

Extract the marker information from the `full_column`, which corresponds to full column in big dataframe.

In [ ]:
# Use regular expressions (regex) to isolate the part of the field that
# begins (^) with an alphanumeric value (W), and ends with an underscore (_)
# '$' is end of line
intensities['marker'] = intensities['full_column'].str.extract(r'([^\W_]+)')
# convert to lowercase
intensities['marker_lower'] = intensities['marker'].str.lower()

In [ ]:
# Subset the intensities df to exclude any column pertaining to DAPI
intensities = intensities.loc[intensities['marker_lower'] != 'dapi']

Now merge the `intensities` and `exp_df` together to create `metadata`

In [ ]:
metadata = pd.merge(exp_df, intensities, how = 'left',
                   left_on = 'target_lower',right_on = 'marker_lower')
metadata = metadata.drop(columns = ['marker_lower'])

# Target is the capitalization from the Exposure_Time.csv
# target_lower is Target in all caps
# marker is the extracted first component of the full column in segmentation data, with corresponding capitalization

Add a column to signify marker target location.

In [ ]:
# Use a lambda to determine segmented location of intensity marker column and update metadata accordingly
# This function determines what the location of the marker is in the cell
# It looks for 'cytoplasm', 'cell',' or 'nucleus' string inside the 
# 'full_column' column of a given row, and returns the identifyied
# area of 'unknown' if none of them
def add_metadata_location(row):
    fc = row['full_column'].lower()
    if 'cytoplasm' in fc and 'cell' not in fc and 'nucleus' not in fc:
        return 'cytoplasm'
    elif 'cell' in fc and 'cytoplasm' not in fc and 'nucleus' not in fc:
        return 'cell'
    elif 'nucleus' in fc and 'cell' not in fc and 'cytoplasm' not in fc:
        return 'nulceus'
    else:
        return 'unknown'

# apply the function
metadata['location'] = metadata.apply(
    lambda row: add_metadata_location(row), axis = 1)

A peek at our `metadata` dataframe:

In [ ]:
metadata.head()

Save this data structure to the metadata folder.

In [ ]:
# don't want to add color in because that's better off treating color the same for round, channel, and sample
filename = "marker_intensity_metadata.csv"
filename = os.path.join(metadata_dir, filename)

metadata.to_csv(filename, index = False)

### Import sample metadata if applicable

In [ ]:
filename = "ROI_Map.csv"
filename = os.path.join(metadata_dir, filename)

sample_metadata = pd.read_csv(filename)

In [ ]:
# Verify file imported correctly

# Verify size
# This part is wrapped in a try/except block because 
# it wasn't working on the PC workstation, but worked
# on MG's personal PC laptop and department loaner MacBook
try:
    verify_line_no(filename, sample_metadata.shape[0] + 1)
    print("Ran file length verification.")
except:
    pass

# Headers
print("Assessing whether column headers are as expected.")
expected_headers =['Sample_ID', 'ROI_slide','ROI_index', 'TMA_Core', 'TMA_row',
                   'TMA_column', 'tissue_long', 'tissue_short', 'Replicate', 'Type']
compare_headers(expected_headers, sample_metadata.columns.values, "Imported metadata file")

# Missingness
if exp_df.isnull().any().any():
    print("\nexp_df has null value(s) in row(s):")
    print(sample_metadta[sample_metadata.isna().any(axis=1)])
else:
    print("No null values detected.")

In this case, `sample_metadata` does not need to be merged with any other df and then saved again

## Establish colors to use throughout workflow

#### Channel colors

Channel colors - want colors that are categorical, since Channel is a non-ordered category (yes, they are numbered, but arbitrarily). A categorical color palette will have dissimilar colors. However, it we will typically use a prescribed set of channel colors that are consistent throughout experiments: c2 = green, c3 = orange, c4 = red, c5 = turquoise. The more automated channel color generation will be left below for reference.

In [ ]:
# Get those unique colors
if len(metadata.Channel.unique()) > 10:
    print("WARNING: There are more unique channel values than \
    there are colors to choose from. Select different palette, e.g., \
    continuous palette 'husl'.")
channel_color_values = sb.color_palette("colorblind",n_colors = len(metadata.Channel.unique()))#'HLS'
# chose 'colorblind' because it is categorical and we're unlikely to have > 10

print("Unique channels are:", metadata.Channel.unique())
# Display those unique colors
sb.palplot(sb.color_palette(channel_color_values))


Store in a dictionary

In [ ]:
channel_color_dict = dict(zip(metadata.Channel.unique(), channel_color_values))

channel_color_dict

Let's choose our channel colors instead. We can use the function `matplotlib.colors.to_rbg(c)`, where `c` is a word color name, to convert to the (r, g, b) tuple needed for the workflow. At the top of the script, we imported `matplotlib.colors` as `mplc`, so we can save time and type out simply `mplc.to_rgb(c)` shorthand when using this function. Note that if you use any of the xkcd color survey colors (https://xkcd.com/color/rgb/), you will need to call these specify these as 'xkcd:colorname'.

I will demonstrate a couple of different ways of doing changing the colors we generated above, so the user can expand on the examples as necessary. We are holding all of our color information in several instances of a data structure called a dictionary. https://docs.python.org/3/library/stdtypes.html#typesmapping

Dictionaries are a way to store an unordered collection of items where each is composed of a key-value mapped pair. In the case of this workflow, each color dictionary has a string identifying the specific thing to be colored, e.g., 'c2', 'TMA', 'cluster1', or 'r5', and the corresponding value is a three-float tuple (r, g, b) that is the color of that thing. With dictionaries, we can remove an key-value pair, add a new key-value pair, or overwrite an existing key-value pair whenever we want. Keys can be many things, but often you will see them as a string. Values can be strings, lists, other dictionaries (as seen below for the heatmaps), etc. Nested dictionaries can be complicated to intuit, but they can be a good way to associate a bunch of information together easily, coding-wise. Keys are not ordered within a dictionary.

In [ ]:
# get a new color for a channel, overwrite/replace the original channel color in the dictionary

c2_color = "green"
c2_color = mplc.to_rgb("green")
print("Our new color in rbg form is " + str(c2_color) + ".")

print("Before replacement, c2 in the dictionary is: " + str(channel_color_dict['c2']))

# Replace value
channel_color_dict['c2'] = c2_color
print("After replacement, c2 in the dictionary is: " + str(channel_color_dict['c2']))

In [ ]:
# Here is how you delete an item from a dictionary

print("Keys in the channel color dictionary are: " + str(channel_color_dict.keys()))

# If we try to remove an existing key, we will get an error
if 'c2' in channel_color_dict.keys():
    print("'c2' is in the dictionary. Removing now.")
    channel_color_dict.pop('c2')
    
print("Keys in the channel color dictionary are: " + str(channel_color_dict.keys()))

In [ ]:
## Add in a new item
print("Keys in the channel color dictionary are: " + str(channel_color_dict.keys()))
print("Adding in 'c2'...")
channel_color_dict['c2'] = c2_color
print("Keys in the channel color dictionary are: " + str(channel_color_dict.keys()))


In [ ]:
## Let's finish the dictionary now

channel_color_dict['c2'] = mplc.to_rgb('green')
channel_color_dict['c3'] = mplc.to_rgb('orange')
channel_color_dict['c4'] = mplc.to_rgb('red')
channel_color_dict['c5'] = mplc.to_rgb('turquoise')

In [ ]:
## And display the colors so we can see them

# Instead of querying the dictionary to get each of our colors, THEN putting those colors in a list,
# THEN feeding that list into the palplot/color_palette code as above, I will condense these steps
# together. Here we are accessing each (r,g,b) color value in the dictionary using the key.
print(['c2','c3','c4','c5'])
sb.palplot(sb.color_palette(
    [channel_color_dict['c2'],channel_color_dict['c3'],channel_color_dict['c4'],channel_color_dict['c5']]))


#### Round colors

Round colors - want colors that are sequential, since Round is an ordered category. We can still generate colors that are easy to distinguish. Also, many of the categorical palettes cap at at about 10 or so unique colors, and repeat from there. We do not want any repeats!

In [ ]:
round_color_values = sb.cubehelix_palette(
    len(metadata.Round.unique()), start=1, rot= -0.75, dark=0.19, light=.85, reverse=True)
#round_color_values = sb.color_palette("cubehelix",n_colors = len(metadata.Round.unique()))
# chose 'cubehelix' because it is sequential, and round is a continuous process
# each color value is a tuple of three values: (R, G, B)
print(metadata.Round.unique())

sb.palplot(sb.color_palette(round_color_values))

## TO-DO: write what these parameters mean

Store in a dictionary

In [ ]:
round_color_dict = dict(zip(metadata.Round.unique(), round_color_values))

for k,v in round_color_dict.items():
    round_color_dict[k] = np.float64(v)

#### Sample colors

Sample colors - want colors that are neither sequential nor categorical. Categorical would be ideal if we could generate an arbitrary number of colors, but I do not think that we can. Hense, we will choose `n` colors from a continuous palette. First we will generate the right number of colors. Later, we will assign TMA samples to gray.

In [ ]:
# Get those unique colors
color_values = sb.color_palette("husl",n_colors = len(ls_samples))#'HLS'
# each color value is a tuple of three values: (R, G, B)

# Display those unique colors
sb.palplot(sb.color_palette(color_values))

Generate enough gray shades for all TMA samples in dataset.

In [ ]:
# Get list of all TMA samples
# by looking for substring 'TMA' in all unique Sample_ID values
TMA_samples = [s for s in df.Sample_ID.unique() if 'TMA' in s]

# Now make a list of unique gray shades,
# whose length equals the length of the list above
TMA_color_values = sb.color_palette(n_colors = len(TMA_samples),palette = "gray")

# Show the gray color(s) to the user
sb.palplot(sb.color_palette(TMA_color_values))

#### Store in a dictionary

In [ ]:
# Now we will create a dictionary to hold this information
# Here we are mapping the unique Sample_ID values in df
# (note that sorted() ensures they are in alphabetical
# order) with the color_values list we derived above.
# This list does NOT have our TMA gray(s) in it.
# After we associate the two groups of items together
# with zip, we turn it into a dictonary: key = Sample_ID,
# value = color for that Sample_ID
sample_color_dict = dict(zip(
    sorted(df.Sample_ID.unique()), color_values
            ))

# Edit our dictioanry
# Replace all TMA samples' colors with gray by
# iterating through all keys in sorted order
# and replacing the color with a gray one. We are
# moving through our list of gray colors using our
# index 'i', so that each TMA gets a different gray.
i = 0
for key in sorted(sample_color_dict.keys()):
    if 'TMA' in key:
        sample_color_dict[key] = TMA_color_values[i]
        i +=1

In [ ]:
sample_color_dict

Look at the (r,g,b) values of the colors above. Any TMA sample should have r ~= g ~= b.

Display the colors:

In [ ]:
print("Our samples and corresponding colors are:")
print([key for key in sorted(sample_color_dict.keys())])
sb.palplot(sb.color_palette([sample_color_dict[key] for key in sorted(sample_color_dict.keys())]))

### Save color information (mapping and legend) to metadata directory

In [ ]:
# let's look at the metadata again...
metadata.head()


Add in the color information in both RGB (range 0-1) and hex values, for use in visualizations

In [ ]:
metadata['round_color'] = metadata.apply(lambda row: round_color_dict[row['Round']], axis = 1)
metadata['channel_color'] = metadata.apply(lambda row: channel_color_dict[row['Channel']], axis = 1)

In [ ]:
# This function takes in a dictionary cd, a column_name string
# and returs a dataframe. This df has the information that was
# in the dictionary--'rgb' is the (fl, fl, fl) tuple corresponding
# to the color names given as the cd keys, an 'hex' is the corresponding
# hexademical value.
def color_dict_to_df(cd, column_name):
    df = pd.DataFrame.from_dict(cd, orient = 'index')
    df['rgb'] = df.apply(lambda row: (np.float64(row[0]), np.float(row[1]), np.float64(row[2])), axis = 1)
    df = df.drop(columns = [0,1,2])
    df['hex'] = df.apply(lambda row: mplc.to_hex(row['rgb']), axis = 1)
    df[column_name] = df.index
    return df

Sample

In [ ]:
# Create dataframe
color_df = color_dict_to_df(sample_color_dict, "Sample_ID")
color_df.head()

# Save to file in metadatadirectory
filename = "sample_color_data.csv"
filename = os.path.join(metadata_dir, filename)
color_df.to_csv(filename, index = False)

In [ ]:
# Legend of sample info only

g  = plt.figure(figsize = (1,1)).add_subplot(111)
g.axis('off')
handles = []
# To change the order of items on the legend, do
# for item in [item1, item2, item3]:
for item in sorted(sample_color_dict.keys()):
        h = g.bar(0,0, color = sample_color_dict[item],
                  label = item, linewidth =0)
        handles.append(h)
first_legend = plt.legend(handles=handles, loc='upper right', title = 'Sample'),
                            # bbox_to_anchor=(10,10), 
                             #       bbox_transform=plt.gcf().transFigure)

# Save the legend to a file
filename = "Sample_legend.png"
filename = os.path.join(metadata_images_dir, filename)
plt.savefig(filename, bbox_inches = 'tight')

Channel

In [ ]:
# Create dataframe
color_df = color_dict_to_df(channel_color_dict, "Channel")
color_df.head()

# Save to file in metadatadirectory
filename = "channel_color_data.csv"
filename = os.path.join(metadata_dir, filename)
color_df.to_csv(filename, index = False)

In [ ]:
# Legend of channel info only

g  = plt.figure(figsize = (1,1)).add_subplot(111)
g.axis('off')
handles = []
# To change the order of items on the legend, do
# for item in [item1, item2, item3]:
for item in sorted(channel_color_dict.keys()):
        h = g.bar(0,0, color = channel_color_dict[item],
                  label = item, linewidth =0)
        handles.append(h)
first_legend = plt.legend(handles=handles, loc='upper right', title = 'Channel'),
                            # bbox_to_anchor=(10,10), 
                             #       bbox_transform=plt.gcf().transFigure)

# Save the legend to a file
filename = "Channel_legend.png"
filename = os.path.join(metadata_images_dir, filename)
plt.savefig(filename, bbox_inches = 'tight')

Round

In [ ]:
# Create dataframe
color_df = color_dict_to_df(round_color_dict, "Round")
color_df.head()

# Save to file in metadatadirectory
filename = "round_color_data.csv"
filename = os.path.join(metadata_dir, filename)
color_df.to_csv(filename, index = False)

In [ ]:
# Legend of round info only

round_legend  = plt.figure(figsize = (1,1)).add_subplot(111)
round_legend.axis('off')
handles = []
# To change the order of items on the legend, do
# for item in [item1, item2, item3]:
for item in round_color_dict.keys():
        h = round_legend.bar(0,0, color = round_color_dict[item],
                  label = item, linewidth =0)
        handles.append(h)
first_legend = plt.legend(handles=handles, loc='upper right', title = 'Round'),
                            # bbox_to_anchor=(10,10), 
                             #       bbox_transform=plt.gcf().transFigure)

# Save the legend to a file
filename = "Round_legend.png"
filename = os.path.join(metadata_images_dir, filename)
plt.savefig(filename, bbox_inches = 'tight')

## EDA scatterplot

Scatterplot of nucleus size by nucleus roundness, colored by sample

This was not working on my computer, probably due to the size of the data. Let's run this chunk using just a subset of the data. Here, we will want the subset to maintain the same proportion of cells for each Sample_ID as we had in the original dataframe.

In [ ]:
subset_row_count = 10000

In [ ]:
subset_df = create_subset(df, 'Sample_ID', subset_row_count, 'original')

How many lines for each sample ID are in our subset df?

In [ ]:
subset_df['Sample_ID'].value_counts().sort_index()

How do the proportions of cells in the original and subset dfs compare?

In [ ]:
df['Sample_ID'].value_counts().sort_index()/df.shape[0]

In [ ]:
subset_df['Sample_ID'].value_counts().sort_index()/subset_df.shape[0]

Perform the plotting.

In [ ]:
#By sample ID only

# initiate figure
fig = go.Figure()
title = 'Nucleus size by nucleus roundess by Sample ID'

# plot each trace separately
for sample in ls_samples:
    fig.add_trace(go.Scatter(
        x = subset_df.loc[subset_df['Sample_ID']==sample,'Nucleus_Roundness'],
        y = subset_df.loc[subset_df['Sample_ID']==sample,'Nucleus_Size'],
        mode = 'markers',
        name = sample,
        marker=dict(
            color='rgb' + str(sample_color_dict[sample])),
            showlegend = True
        
    ))
    

# Update figure for aesthetic details
fig.update_layout(title = title, plot_bgcolor = 'white')
fig.update_xaxes(title_text = "Nucleus roundness", linecolor = 'black')
fig.update_yaxes(title_text = "Nucleus size", linecolor = 'black')

# Output
#plot(fig) # plot generates in new Chrome tab
# Write to file
filename = os.path.join(output_images_dir, title.replace(" ","_") + ".png")
fig.write_image(filename)

## Initial heatmap

We will only be plotting ~10k cells in the interest of time/computing resources. We want these 10k lines in our original df to be sampled randomly, without replacement, with the caveat that the proportions of all samples in the data are equal to each other (unless a particular sample does not have enough corresponding lines for the desired final df size). If the size of the dataframe is > 10k rows, then we will proceed with the entire dataset.

In [ ]:
subset_row_count = 10000

In [ ]:
subset_df = create_subset(df, 'Sample_ID', subset_row_count, 'equal')

How many lines for each sample ID are in our subset df?

In [ ]:
subset_df['Sample_ID'].value_counts().sort_index()

How do the proportions of cells in the original and subset dfs compare?

In [ ]:
df['Sample_ID'].value_counts().sort_index()/df.shape[0]

In [ ]:
subset_df['Sample_ID'].value_counts().sort_index()/subset_df.shape[0]

### Get data structures to map colors to columns and rows...

## Row colors

For the row colors, we essentially just need to map the information in a given feature to the colors that correspond to that value in the right color dictionary. For example, it might be sample_3, sample_3, sample_4, , so we need the row colors to be (1, 1, 1), (1, 1, 1), (0, 0.25, 0.6). These are the initialy colors--if we are clustering rows or columns, the labels will still match the data with which they're associated.

In [ ]:
row_sample_colors = subset_df.Sample_ID.map(sample_color_dict)

row_sample_colors[1:5]

## Column rows

For column rows, matching up the information in each column with the appropriate color is more difficult. 

In [ ]:
# Here, we want to translate marker columns to their corresponding channel information,
# and then match that up with the right color, as with row columns

# First, we merge the (L) non-intensity column values, transformed into a dataframe,
# with the metadata df (R), matching on the "0" column present in the L,
# which is the only column in there, with the "full_column" (aka df header name)
# column in the R, only including all cases where there is a match and any unmatched
# L cases ('both' [?] would be only cases where ther is is a match, and 'right' would
# be cases with a match and any unmatched R columns).
column_channel_colors = pd.merge(pd.DataFrame(pd.Series(
    subset_df.loc[:,~subset_df.columns.isin(not_intensities)].columns.values)), 
                  metadata, how = 'left',
         left_on = 0, right_on = 'full_column'
                                # From that resulting df, extract the '0' and 'Channel' objects,
                                # then only 'Channel', then map to the right colors
                                )[[0,'Channel']]['Channel'].map(channel_color_dict)

# Set the index to be the names of the colors. There is only one column, and that is the corresponding
# colors
column_channel_colors.index = subset_df.loc[:,~subset_df.columns.isin(not_intensities)].columns.values

column_channel_colors.head()

In [ ]:
# Here, we want to translate marker columns to their corresponding round information,
# and then match that up with the right color, as with row columns

# First, we merge the (L) non-intensity column values, transformed into a dataframe,
# with the metadata df (R), matching on the "0" column present in the L,
# which is the only column in there, with the "full_column" (aka df header name)
# column in the R, only including all cases where there is a match and any unmatched
# L cases ('both' [?] would be only cases where ther is is a match, and 'right' would
# be cases with a match and any unmatched R columns).
column_round_colors = pd.merge(pd.DataFrame(pd.Series(
    subset_df.loc[:,~subset_df.columns.isin(not_intensities)].columns.values)), 
                  metadata, how = 'left',
         left_on = 0, right_on = 'full_column'
                              # From that resulting df, extract the '0' and 'Channel' objects,
                                # then only 'Channel', then map to the right colors
                              )[[0,'Round']]['Round'].map(round_color_dict)

# Set the index to be the names of the colors. There is only one column, and that is the corresponding
# colors
column_round_colors.index = subset_df.loc[:,~subset_df.columns.isin(not_intensities)].columns.values

column_round_colors.head()

### Annotations data structure

In [ ]:
# Create data structure to hold everything we need for row/column annotations
# annotations is a dictionary
## IMPORTANT - if you use 'annotations', it MUST have both 'rows' and 'cols'
## objects inside. These can be empty lists, but they must be there!
annotations = {}

# create a data structure to hold everything we need for only row annotations
# row_annotations is a list, where each item therein is a dictioary corresponding
# to all of the data pertaining to that particular annotation
# Adding each item (e.g., Sample, then Cluster), one at a time to ensure ordering
# is as anticipated on figure
row_annotations = []
row_annotations.append({'label':'Sample','type':'row','mapping':row_sample_colors,'dict':sample_color_dict,
                        'location':'center left','bbox_to_anchor':(0, 0.5)})
# Add all row information into the annotations dictionary
annotations['rows'] = row_annotations


# Now we repeat the process for column annotations
col_annotations = []
col_annotations.append({'label':'Round','type':'column','mapping':column_round_colors,'dict':round_color_dict,
                       'location':'upper right','bbox_to_anchor':(1,0.50)})

col_annotations.append({'label':'Column','type':'column','mapping':column_channel_colors,'dict':channel_color_dict,
                       'location':'upper right','bbox_to_anchor':(1,0.75)})
annotations['cols'] = col_annotations

#### Actually plot the heatmap

In [ ]:
heatmap_function(
    data = subset_df.loc[:,~subset_df.columns.isin(not_intensities)],
    title = "Initial dataframe",
    # define method, metric, and color map
    method = 'ward', metric = 'euclidean',cmap = 'coolwarm',
    # colorbar info (legend coloring of main plot) 
    cbar_kws = {'label':'Intens.'},
    # xticklabels - want to have the nicknames instead of full names,
    # so we translate from full to short names; we also only want to include
    # non_intensity columns, to match the data we fed into under 'data'
    xticklabels = [full_to_short_names[name] for name in 
                     subset_df.loc[:,
                                 ~subset_df.columns.isin(not_intensities)].columns.values],
    # where to save the df
    save_loc = output_images_dir,
    # how to cluster on rows and columns
    row_cluster = True, col_cluster = True,
    # provide the dictionary of row and column coloring information
    # and legend information, as established above.
    annotations = annotations
          )

### Bar plot of count of all cells in all samples - no filtering yet

In [ ]:
# Get counts for each Sample_ID, sorted by Sample_ID
counts = pd.DataFrame(df.Sample_ID.value_counts()).sort_index()

# rename Sample_ID to counts
counts = counts.rename(columns = {'Sample_ID':'counts'})
# add Sample_ID back in, as what's currently the index
counts['Sample_ID'] = counts.index
# add 'color', which is derived from the row's Sample_ID fed into the right
# color dictionary
counts['color'] = counts.apply(lambda row: sample_color_dict[row['Sample_ID']], axis = 1)
counts.head()

In [ ]:
ls_samples

In [ ]:
# By sample ID only

# establish figure
fig = go.Figure()
title = 'Initial Cell counts by Sample ID'

# Changing the ordering of the bars is a easy as iterating through a list
# with the samples in a different order! For example, this order below:
#for sample in ['TMA', 'GZ7.2', 'GZ10.3', 'GZ7.1', 'GZ10.2', 'GZ10.1', 'GZ6']:
for sample in ls_samples:
    # add trace for each sample
    fig.add_trace(go.Bar(
        x=counts.loc[counts['Sample_ID']==sample,'Sample_ID'], 
        y = counts.loc[counts['Sample_ID']==sample,'counts'],
        text = counts.loc[counts['Sample_ID']==sample,'counts'], 
        textposition='outside',
        marker=dict(
            color='rgb' + str(sample_color_dict[sample])),
            showlegend = False
        
    ))
    
# update aesthetic parameters
fig.update_layout(title = title, plot_bgcolor = 'white')
fig.update_xaxes(title_text = "Sample ID", linecolor = 'black')
fig.update_yaxes(title_text = "Cell count", linecolor = 'black')

# Display plot
#plot(fig)
filename = os.path.join(output_images_dir, title.replace(" ","_") + ".png")
fig.write_image(filename)


## PCA

This is how you might save data for the PCA, if you'd like to.

In [ ]:
## for PCA
filename = "weewin_PCA_test.csv"
filename = "repro_PCA_test.csv"
filename = "gz_PCA_test.csv"
df.to_csv(filename, index = False)


### Drop any other rows or columns we want to before saving data

In [ ]:
# Let's take a look
df.columns.values

For the sake of example, I will operate on a copy of df, called df_copy

In [ ]:
# You MUST do df.copy()
# 'df_copy = df' would essentially 
# give you two different names for the
# SAME dataframe, so operating on one
# would also operate on the other
df_copy = df.copy()

#### Operate on entire rows or columns

In [ ]:
# Drop columns
my_cols = []
df_copy = df_copy.drop(columns = my_cols)

In [ ]:
# Keep only specific columns (explained below)
my_cols = []
my_cols = df.columns.values
df_copy = df_copy.loc[:,my_cols]


#### Operate on rows and columns using filtering criteria

In [ ]:
# Keep only certain rows based off of criteria

# use df.loc[] to filter
# df.loc[rows,columns]
# df.loc[:,certain_cols] --> keep all rows ':', only certain cols
# df.loc[certain_rows,:] --> keep only certain row, all cols ':'

# Say we only want certain values for Sample_ID
print(df_copy.Sample_ID.unique())
keep = ['TMA','GZ6']
#keep = ['TMA1.1','TMA1.2','TMA1.3','TMA2.1','TMA2.2','TMA2.3']
df_copy = df_copy.loc[df_copy['Sample_ID'].isin(keep),:]
print(df_copy.Sample_ID.unique())

In [ ]:
# Filter on multiple criteria
# '&' or 'and'
# '|' or 'or'
# you MUST have parentheses around each logic expression!
df_copy = df_copy.loc[
    (df_copy['Sample_ID'].isin(['TMA1.1','TMA1.2','TMA1.3'])) \
    ## backslash above used to break line for readability, but tell Python to act like it's all one line
        | (df_copy['Sample_ID'].isin(['TMA2.1','TMA2.2','TMA2.3'])) , :]
print(df_copy.Sample_ID.unique())

In [ ]:
# Remove rows based off of certain criteria
# note the negating tilde '~'!

df_copy = df_copy.loc[
    (~df_copy['Sample_ID'].isin(['TMA1.1','TMA1.2','TMA1.3'])) \
    ## backslash above used to break line for readability, but tell Python to act like it's all one line
        & (~df_copy['Sample_ID'].isin(['TMA2.1','TMA2.2','TMA2.3'])),:]
print(df_copy.Sample_ID.unique())

## include example for cell types: cancer, stroma, immune

### Save the data by Sample_ID

In [ ]:
# Check for existence of output file first
for sample in ls_samples:
    filename = os.path.join(output_data_dir,  sample + "_" + step_suffix + ".csv")
    if os.path.exists(filename):
        print("File by name "+filename+" already exists.")

In [ ]:
# Save output files
for sample in ls_samples:
    df_save = df.loc[df['Sample_ID'] == sample,:]
    filename = os.path.join(output_data_dir,  sample + "_" + step_suffix + ".csv")
    df_save.to_csv(filename, index = True)
